# Put All Files Together

In [ ]:
import pandas as pd
import os
from glob import glob
from collections import defaultdict

# 🔧 Set the root directory where the metrics subdirs exist
metrics_root_dir = "/media/mujtaba/DATA/nick/UnitTestExamples/UnitTestGenEvaluation/metrics"  # <--- change this

# Define subdirectories inside the root
base_dirs = {
    "bleu": os.path.join(metrics_root_dir, "BLEU"),
    "rsr": os.path.join(metrics_root_dir, "RSR"),
    "csr": os.path.join(metrics_root_dir, "CSR")
}

# Map base filename (without metric suffix) to its files
file_map = defaultdict(dict)

# Helper: strip suffix like "_CSR", "_RSR", or "_bleu"
def strip_metric_suffix(filename):
    for metric in ["_CSR", "_RSR", "_bleu"]:
        if filename.endswith(metric):
            return filename.replace(metric, "")
    return filename

# Step 1: Build file map
for metric, dir_path in base_dirs.items():
    files = glob(os.path.join(dir_path, "*.csv"))
    for f in files:
        base_name = os.path.splitext(os.path.basename(f))[0]
        key = strip_metric_suffix(base_name)
        file_map[key][metric] = f

# Step 2: Process each key
rows = []

for key in sorted(file_map.keys()):
    row = {"model_prompt": key}

    # BLEU
    if "bleu" in file_map[key]:
        df_bleu = pd.read_csv(file_map[key]["bleu"])
        row["BLEU"] = df_bleu["BLEU"].mean()

    # RSR
    if "rsr" in file_map[key]:
        df_rsr = pd.read_csv(file_map[key]["rsr"])
        row["RSR"] = df_rsr["RunSuccess"].mean() * 100

    # CSR
    if "csr" in file_map[key]:
        df_csr = pd.read_csv(file_map[key]["csr"])
        row["CSR"] = df_csr["CompileSuccess"].mean() * 100

    rows.append(row)

# Final table
summary_df = pd.DataFrame(rows)
summary_df = summary_df[["model_prompt", "BLEU", "RSR", "CSR"]]  # Optional column order

# Show or save
print(summary_df)
#summary_df.to_csv("summary_metrics.csv", index=False)

similarity_root_dir = "/media/mujtaba/DATA/nick/UnitTestExamples/data/results/metrics"
similarity_dirs = [
    "CodeLlama-7b-Instruct-hf__few_shot_first_prompts",
    "CodeLlama-7b-Instruct-hf__zero_shot_first_prompts",
    "starcoder2-3b__few_shot_first_prompts",
    "starcoder2-3b__zero_shot_first_prompts",
    "DeepSeek-Coder-V2-Lite-Base__few_shot_first_prompts",
    "DeepSeek-Coder-V2-Lite-Base__zero_shot_first_prompts",
    "codet5p-2b__few_shot_first_prompts",
    "codet5p-2b__zero_shot_first_prompts"

]

# === Collect similarity values by model_prompt name
similarity_scores = {}

for model_prompt in similarity_dirs:
    sim_path = os.path.join(similarity_root_dir, model_prompt, "ast_metrics.csv")

    try:
        df = pd.read_csv(sim_path)
        sim_avg = df["similarity"].mean()
        print(f"{model_prompt}: {sim_avg}")
        similarity_scores[model_prompt] = sim_avg
    except Exception as e:
        print(f"Skipping {sim_path} due to error: {e}")
        similarity_scores[model_prompt] = None

# === Merge into summary_df
summary_df["Edit_Similarity"] = summary_df["model_prompt"].map(similarity_scores)

# === Save output
print(summary_df)
summary_df.to_csv("summary_metrics.csv", index=False)



                                        model_prompt       BLEU        RSR  \
0   CodeLlama-7b-Instruct-hf__few_shot_first_prompts  56.432857   9.756098   
1  CodeLlama-7b-Instruct-hf__zero_shot_first_prompts  25.315455  46.341463   
2              starcoder2-3b__few_shot_first_prompts  58.624118  12.195122   
3             starcoder2-3b__zero_shot_first_prompts  35.710000  35.975610   

         CSR  
0  32.317073  
1  46.341463  
2  31.097561  
3  35.975610  
CodeLlama-7b-Instruct-hf__few_shot_first_prompts: 0.17362195121951218
CodeLlama-7b-Instruct-hf__zero_shot_first_prompts: 0.1583109756097561
starcoder2-3b__few_shot_first_prompts: 0.1839329268292683
starcoder2-3b__zero_shot_first_prompts: 0.16599999999999998
                                        model_prompt       BLEU        RSR  \
0   CodeLlama-7b-Instruct-hf__few_shot_first_prompts  56.432857   9.756098   
1  CodeLlama-7b-Instruct-hf__zero_shot_first_prompts  25.315455  46.341463   
2              starcoder2-3b__few_shot_fir